In [ ]:
CSI_Path = ""
Pos_Path = "" 
#File Paths

In [ ]:
import h5py
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from pickle import dump

Load Dataset

In [ ]:
f = h5py.File(CSI_Path, "r")
H_Re, H_Im = f['h_Estimated'][:]
f.close()
f = h5py.File(Pos_Path , "r")
pos = f["r_Position"][:]
f.close()

In [ ]:
H_Mag = H_Re**2 + H_Im**2
H_Mag = np.sqrt(H_Mag)
H_Mag = H_Mag.T
pos = pos.T

Creating a test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(H_Mag, pos, test_size=0.1, random_state=42)

Channel charting using PCA

In [ ]:
pca = PCA(66)

In [ ]:
X_T = X_train.reshape(-1,924)
X_T = pca.fit_transform(X_T)
X_T = X_T.reshape(X_train.shape[0] , X_train.shape[1],66)
X_T = X_T.reshape(X_train.shape[0],-1,1)

In [ ]:
dump(pca , open("PCA_Unsupervised.sav" , "wb"))

Estimating postitions from obtained channel charts

In [ ]:
import numpy as np
from tensorflow.keras.models import Model,Sequential , load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense , Dropout, Conv1D ,Flatten

from tensorflow.keras.optimizers import Adam, SGD
def classifier():
    ''' Deep Convolutional NN for position estimation'''
    model = Sequential()
    model.add(Conv1D(16,3,input_shape=(1056,1)))
    model.add(Flatten())
    model.add(Dropout(0.03))
    model.add(Dense(1024 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(256 ,activation = 'relu' ))
    model.add(Dense(128 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(64 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(32 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(16 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(6 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer = Adam(7e-4)) 
    return model

In [ ]:
model = classifier()
model.summary()

In [ ]:
'''Training Model'''
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience = 80, verbose=1)
checkpointer = ModelCheckpoint('best_PCA_CNN.h5', verbose=1, save_best_only=True)
hist = model.fit(X_T,y_train , epochs=1000, validation_split=0.1  ,callbacks=[earlystopper, checkpointer])
val_loss = hist.history['val_loss']

Testing

In [ ]:
X_test = X_test.reshape(-1,924)
X_test = pca.transform(X_test)
X_test = X_test.reshape(-1,1056,1)

In [ ]:
from tensorflow.keras.models import Model,Sequential , load_model
best = load_model("Best_PCA_CNN.h5")
out = best.predict(X_test)
error = np.sum((out-y_test)**2 , axis=1)

In [ ]:
np.mean(np.sqrt(error))

In [ ]:
import matplotlib.pyplot as plt
plt.title("Root Mean Squared Error")
plt.hist(np.sqrt(error),50)